In [2]:
SIZE_FACE=48

In [1]:
import cv2
import pandas as pd
import numpy as np
from PIL import Image

In [3]:
FILE_PATH = 'fer2013.csv'
data = pd.read_csv(FILE_PATH)

In [4]:
data.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [5]:
EMOTIONS = ['angry', 'disgusted', 'fearful', 'happy', 'sad', 'surprised', 'neutral']
def emotion_labeller(x):
    d = np.zeros(len(EMOTIONS))
    d[x] = 1.0
    return d

In [6]:
# returns pixel 48*48
def image_resizer(data):
    data_image = np.fromstring(str(data), dtype = np.uint8, sep = ' ').reshape((SIZE_FACE, SIZE_FACE))
    #data_image = Image.fromarray(data_image).convert('RGB')
    #data_image = np.array(data_image)[:, :, ::-1].copy()
    
    #data_image = format_image(data_image)
    return data_image

In [7]:
# return image ..
def image_formater(data):
    #print data
    data_image = np.fromstring(str(data), dtype = np.uint8, sep = ' ').reshape((SIZE_FACE, SIZE_FACE))
    data_image = Image.fromarray(data_image).convert('RGB')
    
    #data_image = format_image(data_image)
    return data_image

In [8]:
labels = []
images = []
pixels=[]
index = 1
total = data.shape[0]
for index,row in data.iterrows():
    emotion = emotion_labeller(row[0])
    pixel = image_resizer(row[1])
    image=image_formater(row[1])
    if image is not None:
        labels.append(emotion)
        images.append(image)
        pixels.append(pixel)

In [9]:
target=data['emotion']

In [11]:
x=np.array(pixels)
x.shape

(35887, 48, 48)

In [13]:
t=[]
for i in range(len(pixels)):
    t.append(x[i].reshape(48*48))


In [14]:
len(t[0])

2304

In [15]:
df=pd.DataFrame(t)
df.head(5)

,0,1,2,3,4,5,6,7,8,9,...,2294,2295,2296,2297,2298,2299,2300,2301,2302,2303
0,70,80,82,72,58,58,60,63,54,58,...,159,182,183,136,106,116,95,106,109,82
1,151,150,147,155,148,133,111,140,170,174,...,105,108,95,108,102,67,171,193,183,184
2,231,212,156,164,174,138,161,173,182,200,...,104,138,152,122,114,101,97,88,110,152
3,24,32,36,30,32,23,19,20,30,41,...,174,126,132,132,133,136,139,142,143,142
4,4,0,0,0,0,0,0,0,0,0,...,12,34,31,31,31,27,31,30,29,30


In [16]:
%matplotlib inline
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from time import time


In [17]:
x_train,x_test,y_train,y_test=train_test_split(df,target,test_size=0.25,random_state=42)
print('x_train : ',x_train.shape)
print('y_train : ',y_train.shape)
print('x_test : ',x_test.shape)
print('y_test : ',y_test.shape)

x_train :  (26915, 2304)
y_train :  (26915,)
x_test :  (8972, 2304)
y_test :  (8972,)


In [ ]:
print("Fitting the classifier to the training set")
t0 = time()
param_grid = {'C': [1e3, 5e3, 1e4, 5e4, 1e5],
              'gamma': [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1], }
clf = GridSearchCV(SVC(kernel='rbf', class_weight='balanced'), param_grid)
clf = clf.fit(x_train, y_train)
print("Best estimator found by grid search:")
print(clf.best_estimator_)
print(clf.score(x_test,y_test))
print("done in %0.3fs" % (time() - t0))

Fitting the classifier to the training set
